### 선수엔트리 데이터 수집

In [1]:
#pip install selenium
#pip install bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import pandas as pd

#### 1. Chrome Driver

In [2]:
# 설치한 Chrome driver 경로 (exe파일)
driver = webdriver.Chrome('D:\Study\chromedriver')

#### 2. Set URL & Function

In [3]:
# full_link = link_head + link + link_tail
url_head = 'http://www.statiz.co.kr/team.php?'
url_tail = '&opt&sopt=4'
url= ['team=KIA&year=2020', # KIA
        'cteam=KIA%2B%ED%95%B4%ED%83%9C&year=2019',
        'cteam=KIA%2B%ED%95%B4%ED%83%9C&year=2018',
        'team=%EC%82%BC%EC%84%B1&year=2020', # 삼성
        'cteam=%EC%82%BC%EC%84%B1&year=2019',
        'cteam=%EC%82%BC%EC%84%B1&year=2018',
        'cteam=%EB%A1%AF%EB%8D%B0&year=2020', # 롯데
        'cteam=%EB%A1%AF%EB%8D%B0&year=2019',
        'team=%EB%A1%AF%EB%8D%B0&year=2018',
        'cteam=%EB%91%90%EC%82%B0%2BOB&year=2020', # 두산
        'cteam=%EB%91%90%EC%82%B0%2BOB&year=2019',
        'cteam=%EB%91%90%EC%82%B0%2BOB&year=2018',
        'team=SK&year=2020', # SK
        'cteam=SK&year=2019',
        'cteam=SK&year=2018',
        'team=LG&year=2020', # LG
        'cteam=LG%2BMBC&year=2019',
        'cteam=LG%2BMBC&year=2018',
        'team=%ED%95%9C%ED%99%94&year=2020', # 한화
        'cteam=%ED%95%9C%ED%99%94%2B%EB%B9%99%EA%B7%B8%EB%A0%88&year=2019',
        'cteam=%ED%95%9C%ED%99%94%2B%EB%B9%99%EA%B7%B8%EB%A0%88&year=2018',
        'team=NC&year=2020', # NC
        'cteam=NC&year=2019',
        'team=NC&year=2018',
        'cteam=%ED%9E%88%EC%96%B4%EB%A1%9C%EC%A6%88%2B%EB%84%A5%EC%84%BC%2B%ED%82%A4%EC%9B%80&year=2020', # 키움
        'cteam=%ED%9E%88%EC%96%B4%EB%A1%9C%EC%A6%88%2B%EB%84%A5%EC%84%BC%2B%ED%82%A4%EC%9B%80&year=2019',
        'team=%EB%84%A5%EC%84%BC&year=2018',
        'team=KT&year=2020', # KT
        'cteam=kt&year=2019',
        'cteam=kt&year=2018']

In [4]:
# extract text from selected html source
def ext(source):
    global txt
    for i in source:
        txt = i.text
    return str(txt)

In [5]:
# button이 있는지 확인
# True : 있음 , False : 없음
def checkEN(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False

#### 3. Data crawling

In [6]:
def crawlingPage(soup, data_all):
    # team name (fixed)
    teams = soup.select("div[class='btn-group']>button>font")
    team = ext(teams)
    
    for i in range(9):
        data = []
        selector = 'div:nth-child(' + str(i + 1) + ') > div >'
        pselector = selector + 'div:nth-child(2) > table > tbody >'

        # team
        data.append(team)

        # date
        dates = soup.select(selector + "div > h3 > b")
        date = ext(dates)
        data.append(date)

        # player (name, position, type)
        for j in range(10):
            names = soup.select(pselector + "tr:nth-child(" + str(j + 2) + ") > td:nth-child(2) > a")
            name = ext(names)
            positions = soup.select(pselector + "tr:nth-child(" + str(j + 2) + ") > td:nth-child(3)")
            position = ext(positions)
            types = soup.select(pselector + "tr:nth-child(" + str(j + 2) + ") > td:nth-child(4)")
            tp = ext(types)
            
            data.append(name)
            data.append(position)
            data.append(tp)

        # match
        matchs = soup.select(selector + 'div:nth-child(4) > table > tbody > tr:nth-child(1) > td > a')
        match = ext(matchs)
        data.append(match)

        # opponent pitcher (name + type)
        pitchers = soup.select(selector + 'div:nth-child(4) > table > tbody > tr:nth-child(2) > td ')
        pitcher = ext(pitchers)
        data.append(pitcher[:-5]) # name
        data.append(pitcher[-3:-1]) # type
        
        data_all.append(data)

In [7]:
import time

data_all = []
progress = 0

for i in range(len(url)):
    # X팀 Y연도 page 첫 페이지 열기
    driver.get(url_head + url[i] + url_tail)
    
    for j in range(20):
        time.sleep(10)
        # load html
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # data crawling
        crawlingPage(soup, data_all)
        
        # check
        progress = ((i * 17 + j)/(30 * 17)) * 100
        if j % 5 == 0:
            print("진행중.... (" + str(round(progress,1)) + "%) / 데이터 : " + str(data_all[-1]))
        
        # checkEN을 이용하여 button 활성화 되어있는지 확인 (last page check)
        if checkEN("//button[contains(text(), '다음')]") == True:
            # go next page (botton click)
            nxt = driver.find_element_by_xpath("//button[contains(text(), '다음')]")
            nxt.send_keys(Keys.ENTER)
        
        # button이 상호작용 불가능 할 경우 10sec 대기후 다시 확인
        elif checkEN("//button[contains(text(), '다음')]") == False:
            break

print('.')
print('.')
print('.')
print('.')
print('Data Crawling Done!')

진행중.... (0.0%) / 데이터 : ['20 KIA', '2020-10-22 ', '최원준', 'CF', 'L', '김선빈', '2B', 'R', '터커', 'RF', 'L', '최형우', 'DH', 'L', '나지완', 'LF', 'R', '김민식', 'C', 'L', '황대인', '1B', 'R', '김태진', '3B', 'L', '박찬호', 'SS', 'R', '가뇽', 'P', '우투', 'KIA 10:4 한화', '장민재', '우투']
진행중.... (1.0%) / 데이터 : ['20 KIA', '2020-08-28 ', '최원준', 'CF', 'L', '터커', 'RF', 'L', '최형우', 'DH', 'L', '나지완', 'LF', 'R', '유민상', '1B', 'L', '김민식', 'C', 'L', '박찬호', 'SS', 'R', '황윤호', '3B', 'R', '홍종표', '2B', 'L', '양현종', 'P', '좌투', 'KIA 11:8 SK', '핀토', '우투']
진행중.... (2.0%) / 데이터 : ['20 KIA', '2020-07-01 ', '김선빈', '2B', 'R', '터커', 'RF', 'L', '최형우', 'DH', 'L', '나지완', 'LF', 'R', '유민상', '1B', 'L', '김호령', 'CF', 'R', '나주환', '3B', 'R', '한승택', 'C', 'R', '박찬호', 'SS', 'R', '임기영', 'P', '우언', '한화 3:4 KIA', '장시환', '우투']
진행중.... (2.9%) / 데이터 : ['20 KIA', '2020-05-05 ', '최원준', 'CF', 'L', '김선빈', '2B', 'R', '터커', 'RF', 'L', '최형우', 'DH', 'L', '장영석', '3B', 'R', '유민상', '1B', 'L', '나지완', 'LF', 'R', '한승택', 'C', 'R', '박찬호', 'SS', 'R', '양현종', 'P', '좌투', '키움 11:2 KI

진행중.... (32.4%) / 데이터 : ['18 롯데', '2018-07-21 ', '전준우', 'LF', 'R', '정훈', '1B', 'R', '손아섭', 'RF', 'L', '이대호', 'DH', 'R', '민병헌', 'CF', 'R', '한동희', '3B', 'R', '신본기', 'SS', 'R', '번즈', '2B', 'R', '안중열', 'C', 'R', '송승준', 'P', '우투', 'SK 4:3 롯데', '김광현', '좌투']
진행중.... (33.3%) / 데이터 : ['18 롯데', '2018-05-23 ', '전준우', 'LF', 'R', '문규현', 'SS', 'R', '손아섭', 'RF', 'L', '이대호', '1B', 'R', '이병규', 'DH', 'L', '정훈', 'CF', 'R', '신본기', '3B', 'R', '번즈', '2B', 'R', '김사훈', 'C', 'R', '레일리', 'P', '좌투', '롯데 4:6 삼성', '장원삼', '좌투']
진행중.... (34.3%) / 데이터 : ['18 롯데', '2018-03-24 ', '민병헌', 'CF', 'R', '손아섭', 'RF', 'L', '전준우', 'LF', 'R', '이대호', 'DH', 'R', '채태인', '1B', 'L', '번즈', '2B', 'R', '한동희', '3B', 'R', '신본기', 'SS', 'R', '나원탁', 'C', 'R', '듀브론트', 'P', '좌투', '롯데 5:6 SK', '켈리', '우투']
진행중.... (35.3%) / 데이터 : ['20 두산', '2020-10-17 ', '허경민', '3B', 'R', '최주환', '2B', 'L', '페르난데스', 'DH', 'L', '김재환', 'LF', 'L', '오재일', '1B', 'L', '박건우', 'RF', 'R', '정수빈', 'CF', 'L', '김재호', 'SS', 'R', '장승현', 'C', 'R', '최원준', 'P', '우언', '두산 3:5 키움', 

진행중.... (64.7%) / 데이터 : ['19 LG', '2019-05-25 ', '이천웅', 'CF', 'L', '오지환', 'SS', 'L', '김현수', 'LF', 'L', '조셉', '1B', 'R', '이형종', 'RF', 'R', '박용택', 'DH', 'L', '김민성', '3B', 'R', '유강남', 'C', 'R', '박지규', '2B', 'L', '차우찬', 'P', '좌투', 'LG 6:5 롯데', '톰슨', '우투']
진행중.... (65.7%) / 데이터 : ['19 LG', '2019-03-29 ', '이형종', 'CF', 'R', '정주현', '2B', 'R', '김현수', 'DH', 'L', '조셉', '1B', 'R', '채은성', 'RF', 'R', '유강남', 'C', 'R', '이천웅', 'LF', 'L', '양종민', '3B', 'R', '오지환', 'SS', 'L', '윌슨', 'P', '우투', '롯데 1:2 LG', '레일리', '좌투']
진행중.... (66.7%) / 데이터 : ['18 LG', '2018-09-26 ', '이천웅', 'CF', 'L', '오지환', 'SS', 'L', '서상우', '1B', 'L', '가르시아', 'DH', 'R', '채은성', 'RF', 'R', '양석환', '3B', 'R', '홍창기', 'LF', 'L', '정상호', 'C', 'R', '윤진호', '2B', 'R', '배재준', 'P', '우투', 'LG 2:5 SK', '김광현', '좌투']
진행중.... (67.6%) / 데이터 : ['18 LG', '2018-07-17 ', '이형종', 'CF', 'R', '오지환', 'SS', 'L', '박용택', 'DH', 'L', '김현수', '1B', 'L', '가르시아', '3B', 'R', '채은성', 'RF', 'R', '이천웅', 'LF', 'L', '유강남', 'C', 'R', '정주현', '2B', 'R', '윌슨', 'P', '우투', 'LG 9:3 넥센', 

진행중.... (97.1%) / 데이터 : ['20 키움', '2020-05-05 ', '서건창', '2B', 'L', '김하성', 'SS', 'R', '이정후', 'RF', 'L', '박병호', '1B', 'R', '이택근', 'DH', 'R', '이지영', 'C', 'R', '모터', '3B', 'R', '임병욱', 'CF', 'L', '박준태', 'LF', 'L', '브리검', 'P', '우투', '키움 11:2 KIA', '양현종', '좌투']
진행중.... (98.0%) / 데이터 : ['19 키움', '2019-10-09 ', '서건창', 'DH', 'L', '이정후', 'CF', 'L', '샌즈', 'RF', 'R', '박병호', '1B', 'R', '김하성', 'SS', 'R', '김웅빈', '3B', 'L', '이지영', 'C', 'R', '김규민', 'LF', 'L', '김혜성', '2B', 'L', 'L', 'L', 'L', '키움 2:4 LG', '켈리', '우투']
진행중.... (99.0%) / 데이터 : ['19 키움', '2019-07-30 ', '이정후', 'LF', 'L', '김하성', 'SS', 'R', '샌즈', 'RF', 'R', '박병호', '1B', 'R', '송성문', '3B', 'L', '박동원', 'C', 'R', '임병욱', 'CF', 'L', '이지영', 'DH', 'R', '김혜성', '2B', 'L', '요키시', 'P', '좌투', '키움 4:2 LG', '차우찬', '좌투']
진행중.... (100.0%) / 데이터 : ['19 키움', '2019-05-29 ', '서건창', '2B', 'L', '김하성', 'SS', 'R', '이정후', 'LF', 'L', '박병호', '1B', 'R', '샌즈', 'RF', 'R', '임병욱', 'CF', 'L', '장영석', '3B', 'R', '박동원', 'DH', 'R', '이지영', 'C', 'R', '요키시', 'P', '좌투', 'LG 4:2 키움', '이

In [8]:
# check data length = 144 * 10 * 3 (4320)
print(len(data_all))

4464


In [9]:
# dataframe column 생성
col = ['team', 'date']
for i in range(9):
    col = col + ['player' + str(i + 1), 'position' + str(i + 1), 'type' + str(i + 1)]
col = col + ['pitcher', 'position', 'type', 'result', 'opponent_p', 'type']

# dataframe 생성
df = pd.DataFrame(columns = col, data = data_all)

df.tail()

,team,date,player1,position1,type1,player2,position2,type2,player3,position3,...,type8,player9,position9,type9,pitcher,position,type,result,opponent_p,type
4459,18 kt,2018-03-29,오태곤,LF,R,강백호,DH,L,로하스,CF,...,R,정현,SS,R,류희운,P,우투,kt 7:1 SK,문승원,우투
4460,18 kt,2018-03-28,이진영,DH,L,강백호,LF,L,로하스,CF,...,R,정현,SS,R,금민철,P,좌투,kt 8:5 SK,박종훈,우언
4461,18 kt,2018-03-27,심우준,SS,R,박경수,2B,R,로하스,CF,...,L,장성우,C,R,고영표,P,우언,kt 5:8 SK,산체스,우투
4462,18 kt,2018-03-25,정현,SS,R,오정복,DH,R,로하스,CF,...,L,장성우,C,R,주권,P,우투,kt 1:14 KIA,양현종,좌투
4463,18 kt,2018-03-24,심우준,SS,R,이진영,DH,L,로하스,CF,...,L,장성우,C,R,피어밴드,P,좌투,kt 5:4 KIA,헥터,우투


#### 4. XLSX파일로 저장

In [10]:
df.to_excel("player_entry.xlsx")